# Cours "Géomatique"
### Louis Maritaud
### louis.maritaud@unilim.fr

## Objectifs pédagogiques
- Comprendre la structure d'un GeoDataFrame
- Lire et manipuler des fichiers géographiques
- Maîtriser les bases des systèmes de coordonnées (CRS)
- Créer des cartes simples avec matplotlib


## Partie Théorique

### 1. Introduction à GeoPandas

**Qu'est-ce que GeoPandas ?**
- Extension de Pandas pour les données géographiques
- Même syntaxe que Pandas + colonne spéciale `geometry`
- Supporte shapefile, GeoJSON, GeoPackage, etc.
- Intégration avec matplotlib pour la visualisation

In [ ]:
#Installation et imports
# Installation (à faire une fois)
#pip install geopandas matplotlib contextily

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

# Construction d'un DataFrame classique 
df = pd.DataFrame({
    'ville': ['Paris', 'Lyon', 'Marseille', 'Toulouse', 'Limoges'],
    'population': [2103778, 519127, 877215, 514819, 129754], # Données Wikipédia
    'latitude': [48.8566, 45.7640, 43.2965, 43.6048, 45.849998],
    'longitude': [2.3522, 4.8357, 5.3698, 1.4428, 1.25]
})

print(f"Notre DataFrame classique : \n{df}")

In [ ]:
# Conversion en GeoDataFrame
# Créer des géométries Point à partir des coordonnées → points_from_xy
gdf = gpd.GeoDataFrame(
    df, 
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"  # Système de coordonnées WGS84
)

print(gdf)
print(f"\nColonne geometry :\n{gdf.geometry}")
print(f"\nType de la colonne geometry :{type(gdf.geometry)}")

gdf.plot()
plt.show()


In [ ]:
# Ajout du geojson des régions : 
gdf_regions=gpd.read_file("DATA/regions.geojson") #On lit le geojson et on en fait un geodataframe
ax = gdf_regions.plot(figsize=(20, 10)) # On créé un premier plot à partir du geojson
gdf.plot(ax=ax, color="black") # On ajout à ce premier plot nos points

## Quels types de géométrie pour GéoPandas ? 

- **Point** : un lieu précis (== coordonnée unique)
- **LineString** : une ligne (== succession de points)
- **Polygon** : une surface fermée
- **MultiPoint, MultiLineString, MultiPolygon** : collections de géométries, respectivement plusieurs points ; plusieurs lignes ; plusieurs polygones.

**Exemples :**

In [ ]:
from shapely.geometry import Point, LineString, Polygon
import contextily as ctx 

# Géométries en lon/lat
point = Point(2.3522, 48.8566)  # Paris
ligne = LineString([(2.3522, 48.8566), (4.8357, 45.7640)])  # Paris-Lyon

# Polygone autour de Paris
polygone = Polygon([
    (2.2, 48.8),
    (2.6, 48.8),
    (2.4, 49.0)
])

# Création en EPSG:4326
gdf = gpd.GeoDataFrame(
    {'nom': ['Point', 'Ligne', 'Polygone'],
     'geometry': [point, ligne, polygone]},
    crs="EPSG:4326"
)

gdf

### 3. Systèmes de coordonnées - CRS (Coordinate Reference System)

**Les essentiels à retenir** :

| EPSG Code | Nom | Unité | Usage |
|-----------|-----|-------|-------|
| **4326** | WGS84 | degrés | GPS, **Folium**, coordonnées géographiques |
| **2154** | Lambert 93 | mètres | France, **calculs de distances/aires** |
| **3857** | Web Mercator | mètres | Fonds de carte web (Google Maps, OSM) |

In [ ]:
# Vérifier le CRS actuel
print(gdf.crs)

# Changer de projection (reprojection)
gdf_lambert = gdf.to_crs("EPSG:2154")
print(f"CRS original : {gdf.crs}")
print(f"CRS après reprojection : {gdf_lambert.crs}")

# Comparaison des coordonnées
print("\nCoordonnées en WGS84 (degrés) :")
print(gdf.geometry.head())

print("\nCoordonnées en Lambert 93 (mètres) :")
print(gdf_lambert.geometry.head())

In [ ]:
# Calcul de distance en WGS84 (FAUX !)
paris = Point(2.3522, 48.8566)
limoges = Point(1.25, 45.849998)
distance_deg = paris.distance(limoges)
print(f"Distance en degrés (incorrect) : {distance_deg:.4f}°")

# Calcul de distance en Lambert 93 (CORRECT !)
gdf_villes = gpd.GeoDataFrame({
    'ville': ['Paris', 'Limoges'],
    'geometry': [paris, limoges]
}, crs="EPSG:4326")

gdf_villes_proj = gdf_villes.to_crs("EPSG:2154")
distance_m = gdf_villes_proj.geometry[0].distance(gdf_villes_proj.geometry[1])
print(f"Distance en mètres (correct) : {distance_m:.1f} m")
#Conversion directe des mètres en kilomètres :
print(f"Distance en kilomètres (correct) : {distance_m/1000:.1f} km")
# Résultat attendu : ~345 km

# EXERCICE 1 
Calculez la distance entre Limoges et 

In [ ]:
# Cartes simples avec `.plot()` :
# Carte basique
import matplotlib.pyplot as plt
import contextily as ctx

print(gdf.crs)
# 1. CRS obligatoire
gdf = gdf.to_crs(epsg=3857)

# 2. Une seule figure / un seul axe
fig, ax = plt.subplots(figsize=(12, 10))

# 3. Tracé des données
gdf.plot(
    ax=ax,  # Taille de la figure
    color='red',        # Couleur
    markersize=100,     # Taille des points
    edgecolor='black',  # Contour
    linewidth=2         # Épaisseur du contour
)

# 4. Fond de carte
ctx.add_basemap(
    ax,
    zoom=6
)

# 5. Finalisation
ax.set_axis_off()
plt.title("Principales villes de France")
plt.show()


In [ ]:
import geopandas as gpd
import pandas as pd
import requests
import matplotlib.pyplot as plt

# Charger les contours des départements depuis data.gouv.fr

url = "https://www.data.gouv.fr/fr/datasets/r/90b9341a-e1f7-4d75-a73c-bbc010c7feeb"

r = requests.get(url, verify=True)
r.raise_for_status()
with open("DATA/departements.geojson", "wb") as file :
    file.write(r.content)
with open("DATA/departements.geojson", "r", encoding="utf-8") as file:    
    departements = gpd.read_file(file)
departements=departements.to_crs("EPSG:2154")

# Charger les données de populations prises sur le site de l'INSEE
df_pop=pd.read_csv("DATA/donnees_departements.csv", encoding="utf-8", sep=";")

df_pop["nom"]=df_pop["Département"]
departements=pd.merge(departements, df_pop, on="nom")


# Explorer les données
print("Premières lignes :")
print(departements.head())

print("\nColonnes disponibles :")
print(departements.columns.tolist())

print(f"\nSystème de coordonnées : {departements.crs}")
print(f"Nombre de départements : {len(departements)}")

# Types de géométries
print("\nTypes de géométries :")
print(departements.geometry.type.value_counts())

In [ ]:
# Carte de France simple
fig, ax = plt.subplots(figsize=(12, 12))
departements.plot(ax=ax, edgecolor='black', facecolor='lightblue')
plt.title("Départements français", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

# Afficher quelques noms de départements
print("\nPremiers départements :")
print(departements['nom'].head(15))



# Statistiques sur les géométries
print(f"\nSuperficie totale : {departements.geometry.area.sum()/1_000_000  :.2f} km²")

In [ ]:
# Personnalisation avancée
# Carte avec légende et couleurs selon une variable
fig, ax = plt.subplots(figsize=(12, 10))

departements.plot(
    ax=ax,
    column='PTOT',  # Colorier selon la population
    cmap='YlOrRd',        # Palette de couleurs
    legend=True,
    markersize=200,
    edgecolor='black',
    legend_kwds={'label': "Population"}
)



plt.title("Population des départements français")
plt.axis('off')  # Masquer les axes
plt.tight_layout()
plt.show()

## Palettes de couleurs disponibles pour MatPlotLib 
Visibles via ce lien    
[Palettes de couleurs disponibles avec Matplotlib](https://matplotlib.org/stable/users/explain/colors/colormaps.html)    
Ou cette commande :    

In [ ]:
from matplotlib import colormaps
list(colormaps)

In [ ]:
# Méthode 1 : Filtrer par noms de départements
departements_na = ['Dordogne', 'Gironde', 'Landes', 'Lot-et-Garonne', 
                   'Pyrénées-Atlantiques', 'Deux-Sèvres', 'Vienne', 
                   'Haute-Vienne', 'Charente', 'Charente-Maritime', 
                   'Corrèze', 'Creuse']

nouvelle_aquitaine = departements[departements['nom'].isin(departements_na)]

print(f"Nombre de départements en Nouvelle-Aquitaine : {len(nouvelle_aquitaine)}")

# Visualiser
fig, ax = plt.subplots(figsize=(10, 10))
nouvelle_aquitaine.plot(ax=ax, edgecolor='black', facecolor='#90EE90')
plt.title("Nouvelle-Aquitaine", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

# Méthode 2 : Si un code région existe

# nouvelle_aquitaine = departements[departements['code_region'] == '75']

## A vous 
- Filtrer une autre région (Bretagne, PACA, etc.)
- Compter le nombre de départements par région
- Identifier le plus grand département (en superficie)

In [ ]:
# Solution : Plus grand département
departements_proj = departements.to_crs("EPSG:2154")
departements_proj['superficie_km2'] = departements_proj.geometry.area / 1_000_000

plus_grand = departements_proj.loc[departements_proj['superficie_km2'].idxmax()]
print(f"Plus grand département : {plus_grand['nom']} ({plus_grand['superficie_km2']:.0f} km²)")

## Ajouter des données de population

In [ ]:
# à partir de la jointure déjà faite
# Vérifier s'il y a des valeurs manquantes
if nouvelle_aquitaine['PTOT'].isna().any():
    print("Attention : certains départements n'ont pas de population !")
    print(nouvelle_aquitaine[nouvelle_aquitaine['population'].isna()]['nom'])


## Carte choroplèthe

In [ ]:
# Carte avec gradient de couleurs selon la population
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

nouvelle_aquitaine.plot(
    column='PTOT',     # Variable à visualiser
    cmap='YlOrRd',          # Palette : jaune → orange → rouge
    legend=True,
    edgecolor='black',
    linewidth=0.8,
    ax=ax,
    legend_kwds={
        'label': "Population",
        'orientation': "horizontal",
        'shrink': 0.5
    }
)

plt.title("Population par département - Nouvelle-Aquitaine", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Ajout des départements 
# Calculer les centroïdes (centres) des polygones
nouvelle_aquitaine['centroid'] = nouvelle_aquitaine.geometry.centroid

fig, ax = plt.subplots(1, 1, figsize=(14, 12))

nouvelle_aquitaine.plot(
    column='PTOT', 
    cmap='YlOrRd', 
    legend=True,
    edgecolor='black',
    linewidth=0.8,
    ax=ax
)

# Ajouter les labels au centre de chaque département
for idx, row in nouvelle_aquitaine.iterrows():
    plt.annotate(
        text=row['nom'], 
        xy=(row['centroid'].x, row['centroid'].y),
        horizontalalignment='center',
        fontsize=9,
        weight='bold',
        color='black'
    )

plt.title("Population par département - Nouvelle-Aquitaine", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
## Variables manuelles, bins et classes 

In [ ]:
import matplotlib.colors as mcolors

# Définir des classes
bins = [0, 300000, 500000, 1000000, 2000000]
labels = ['< 300k', '300-500k', '500k-1M', '> 1M']

nouvelle_aquitaine['classe_pop'] = pd.cut(
    nouvelle_aquitaine['PTOT'], 
    bins=bins, 
    labels=labels
)

# Carte avec classes discrètes
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

nouvelle_aquitaine.plot(
    column='classe_pop',
    categorical=True,
    legend=True,
    edgecolor='black',
    linewidth=0.8,
    cmap='RdYlGn_r',  # Inversé : rouge = forte pop
    ax=ax
)

plt.title("Classes de population - Nouvelle-Aquitaine", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Calculer la densité (habitants/km²)
nouvelle_aquitaine_proj = nouvelle_aquitaine.to_crs("EPSG:2154")
nouvelle_aquitaine_proj['superficie_km2'] = (
    nouvelle_aquitaine_proj.geometry.area / 1_000_000
)
nouvelle_aquitaine_proj['densite'] = (
    nouvelle_aquitaine_proj['PTOT'] / 
    nouvelle_aquitaine_proj['superficie_km2']
)

# Cartographier
fig, ax = plt.subplots(figsize=(12, 12))
nouvelle_aquitaine_proj.plot(
    column='densite',
    cmap='cool',
    legend=True,
    edgecolor='black',
    ax=ax,
    legend_kwds={'label': "Densité (hab/km²)"}
)
plt.title("Densité de population")
plt.axis('off')
plt.show()

# Statistiques
print(f"Densité moyenne : {nouvelle_aquitaine_proj['densite'].mean():.1f} hab/km²")
print(f"Département le plus dense : {nouvelle_aquitaine_proj.loc[nouvelle_aquitaine_proj['densite'].idxmax(), 'nom']}")
print(f"Département le moins dense : {nouvelle_aquitaine_proj.loc[nouvelle_aquitaine_proj['densite'].idxmin(), 'nom']}")

In [ ]:
# Filtrer une autre région (ex: Bretagne)
bretagne_depts = ['Côtes-d\'Armor', 'Finistère', 'Ille-et-Vilaine', 'Morbihan']
bretagne = departements[departements['nom'].isin(bretagne_depts)]

# Comparer les populations totales
pop_na = nouvelle_aquitaine['PTOT'].sum()
pop_bretagne = bretagne['PTOT'].sum() if 'PTOT' in bretagne.columns else 0

print(f"Population Nouvelle-Aquitaine : {pop_na:,.0f}")
print(f"Population Bretagne : {pop_bretagne:,.0f}")